In [ ]:
import scipp as sc
import sciline
from essreflectometry.amor import providers as amor
from essreflectometry.reflectometry import providers as reflectometry
from essreflectometry.reflectometry.types import (
    Sample, Reference, Sample, SampleRotation, Filename,
    ThetaData, QBins, NormalizedIOverQ, QStd
)

pipeline = sciline.Pipeline(
    amor + reflectometry,
    params={
        QBins: sc.geomspace(dim='Q', start=0.008, stop=0.075, num=200, unit='1/angstrom'),
        SampleRotation[Sample]: sc.scalar(0.7989, unit='deg'),
        Filename[Sample]: "sample.nxs",
        SampleRotation[Reference]: sc.scalar(0.8389, unit='deg'),
        Filename[Reference]: "reference.nxs",
    }
)

pipeline.visualize((NormalizedIOverQ, QStd))

In [ ]:
normalized = pipeline.compute(NormalizedIOverQ)
normalized.plot(norm='log') + normalized.mean('detector_number').plot(norm='log')


In [ ]:
# Diagnostics plot
(pipeline.compute(ThetaData[Sample])
    .bins.concat('detector_number')
    .hist(
        theta=sc.linspace(dim='theta', start=0.0, stop=1.2, num=165, unit='deg').to(unit='rad'),
        wavelength=sc.linspace(dim='wavelength', start=0, stop=15.0, num=165, unit='angstrom'),
    )
    .plot())


In [ ]:
res = pipeline.compute((NormalizedIOverQ, QStd))
res[QStd].plot()